In [ ]:
import rospy
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from std_msgs.msg import Float64
from dynamixel_sdk import *
import threading
import serial
import re
import pandas as pd

In [ ]:
# For more information on how to set control table of the Dynamixel xm430-w350-R motor,
# please refer to https://emanual.robotis.com/docs/en/dxl/x/xm430-w350/

# Alternatively, you can use Dynamixel Wizard GUI to control the motor,
# https://emanual.robotis.com/docs/en/software/dynamixel/dynamixel_wizard2/

# Please make sure that the power switch on the dynamixel power board is turned on
# to communicate with the dynamixel motor

class Dynamixel_motor:
    def __init__(self, dxl_id, baudrate, device_name):
        self.dxl_id = dxl_id
        self.baudrate = baudrate
        self.device_name = device_name
        self.protocol_version = 2.0

        # Initialize PortHandler instance
        # Set the port path
        # Get methods and members of PortHandlerLinux or PortHandlerWindows
        self.portHandler = PortHandler(self.device_name)

        # Initialize PacketHandler instance
        # Set the protocol version
        # Get methods and members of Protocol1PacketHandler or Protocol2PacketHandler
        self.packetHandler = PacketHandler(self.protocol_version)

        # Open port
        if self.portHandler.openPort():
            print("Succeeded to open the port")
        else:
            print("Failed to open the port")
            print("Press any key to terminate...")
            self.close_port()

        # Set port baudrate
        if self.portHandler.setBaudRate(self.baudrate):
            print("Succeeded to change the baudrate")
        else:
            print("Failed to change the baudrate")
            print("Press any key to terminate...")
            self.close_port()

    def __del__(self):
        print('Destructor called.')
        self.portHandler.closePort()

    def close_port(self):
        self.torque_disable()
        self.portHandler.closePort()
        print('closing port.')

    def ping_motor(self):

        # Try to ping the Dynamixel
        # Get Dynamixel model number
        dxl_model_number, dxl_comm_result, dxl_error = self.packetHandler.ping(self.portHandler, self.dxl_id)
        print(f"dxl_model_number is: {dxl_model_number}")
        print(f"dxl_comm_result is: {dxl_comm_result}")
        print(f"dxl_error is: {dxl_error}")
        print(f"COMM_SUCCESS is {COMM_SUCCESS}")
        if dxl_comm_result != COMM_SUCCESS:
            print("statement 0")
            print("%s" % self.packetHandler.getTxRxResult(dxl_comm_result))
            self.close_port()
        elif dxl_error != 0:
            print("statement 1")
            print("%s" % self.packetHandler.getRxPacketError(dxl_error))
            self.close_port()
        else:
            print("statement 2")
            print("[ID:%03d] ping Succeeded. Dynamixel model number : %d" % (self.dxl_id, dxl_model_number))

    def torque_enable(self):
        TORQUE_ENABLE_REG = 64
        TORQUE_ENABLE = 1
        result, error = self.packetHandler.write4ByteTxRx(self.portHandler, self.dxl_id, TORQUE_ENABLE_REG, TORQUE_ENABLE)
        return

    def torque_disable(self):
        TORQUE_ENABLE_REG = 64
        TORQUE_ENABLE = 0
        result, error = self.packetHandler.write4ByteTxRx(self.portHandler, self.dxl_id, TORQUE_ENABLE_REG, TORQUE_ENABLE)
        return

    def read_operation_mode(self):
        OPERATING_MODE_REG = 11
        data, result, error = self.packetHandler.read1ByteTxRx(self.portHandler, self.dxl_id, OPERATING_MODE_REG)
        print(f"data is: {data}")
        print(f"result is: {result}")
        print(f"error is: {error}")
        return data

    def set_current_position_mode(self):
        OPERATING_MODE_REG = 11
        OPERATION_MODE = 5
        result, error = self.packetHandler.write1ByteTxRx(self.portHandler, self.dxl_id, OPERATING_MODE_REG, OPERATION_MODE)
        return

    def set_goal_current(self, GOAL_CURRENT):
        GOAL_CURRENT_REG = 102
        result, error = self.packetHandler.write2ByteTxRx(self.portHandler, self.dxl_id, GOAL_CURRENT_REG, GOAL_CURRENT)
        return

    def read_present_position(self):
        PRESENT_POSITION_REG = 132
        PRESENT_POSITION, result, error = self.packetHandler.read4ByteTxRx(self.portHandler, self.dxl_id, PRESENT_POSITION_REG)
        return PRESENT_POSITION

    def set_goal_position(self, GOAL_POSITION):
        GOAL_POSITION_REG = 116
        result, error = self.packetHandler.write4ByteTxRx(self.portHandler, self.dxl_id, GOAL_POSITION_REG, GOAL_POSITION)
        return

    def read_present_current(self):
        PRESENT_CURRENT_REG = 126
        PRESENT_CURRENT, result, error = self.packetHandler.read2ByteTxRx(self.portHandler, self.dxl_id, PRESENT_CURRENT_REG)
        return PRESENT_CURRENT


In [ ]:
class Node(object):
    def __init__(self, dxl_motor):
        
        # Run a seperate thread for collecting user input to avoid blocking the main loop
        self.input_thread = threading.Thread(target=self.get_input)
        self.input_thread.daemon = True
        self.input_thread.start()
        
        # Initialise variables
        self.image_array = []
        self.pressure_reading_1 = []
        self.pressure_reading_2 = []
        self.force_reading_1 = []
        self.force_reading_2 = []
        self.command = '0'
        self.take_photo = '0'
        
        # Initialise the Dynamixel motor
        self.dxl_motor = dxl_motor
        self.initialise_motor()
        
        # Initialise the communication to the arduino
        # Please change to the appropriate serial port depending on your setup
        self.ser = serial.Serial('/dev/ttyACM0', 57600, timeout=1)
        self.initialise_arduino()
        
        # Initiate the cv bridge for receiving image from ROS topic
        self.br = CvBridge()
        
        # Node cycle rate (in Hz).
        self.loop_rate = rospy.Rate(1)

        # Publish number command to the Raspberry pi
        self.pub = rospy.Publisher('number_command', Float64, queue_size=10)

        # Subscribe to the image captured by the Raspberry pi
        rospy.Subscriber("/image_capture",Image,self.callback)
    
    def initialise_arduino(self):
        time.sleep(2) # wait for the arduino to initialise
        self.ser.read(200) # Read 200 characters to clear the serial buffer
    
    def untare_read_arduino(self):
        # Number code to send to arduino - 0: clear tare and read, 1: tare and read, 2: read
        self.ser.write(b'0')
        time.sleep(0.1)
        return self.ser.read(40)
    
    def tare_read_arduino(self):
        # Number code to send to arduino - 0: clear tare and read, 1: tare and read, 2: read
        self.ser.write(b'1')
        time.sleep(0.1)
        return self.ser.read(40)
    
    def read_arduino(self):
        # Number code to send to arduino - 0: clear tare and read, 1: tare and read, 2: read
        self.ser.write(b'2')
        time.sleep(0.1)
        return self.ser.read(40)
    
    def initialise_motor(self):
        # Set operation mode to current based position control with a current limit of 50
        self.dxl_motor.set_current_position_mode()
        self.dxl_motor.read_operation_mode()
        self.dxl_motor.set_goal_current(50)
        
    def move_jaw(self, position):
        self.dxl_motor.torque_enable()
        self.dxl_motor.set_goal_position(position)
    
    def disable_jaw(self):
        self.dxl_motor.torque_disable()
    
    def get_input(self):
        while True:
            self.command = input("Enter a character: ")
            time.sleep(1)
    
    def callback(self, msg):
        # Receive image from ROS topic, save the image to a list and display the image
        rospy.loginfo('Image received...')
        self.image = self.br.imgmsg_to_cv2(msg)
        self.image_array.append(self.image)
        plt.imshow(self.image)
        plt.show()
    
    def store_arduino_data(self, data):
        # Split up the data string
        # The expected string format is 'pressure reading 1, pressure reading 2, force reading 1, force reading 2'
        text = re.findall('\d+\.\d+', data.decode("utf-8"))
        print(f'Processed data array is: {text}')
        self.pressure_reading_1.append(float(text[0]))
        self.pressure_reading_2.append(float(text[1]))
        self.force_reading_1.append(float(text[2]))
        self.force_reading_2.append(float(text[3]))
    
    def save_data(self):
        image_name = []
        
        # Please change to the appropriate folder directory depending on your setup
        directory = './Grasp_dataset_2/Grasp_dataset_temp/'
        
        # The first image number name to be saved
        start_number = 0
        
        # Save the list of images to the data directory
        for i in range(0, len(self.image_array)):
            image_name.append('Sq6image' + str(i+start_number).zfill(3) + '.jpg')
            cv2.imwrite(directory +'Sq6image' + str(i+start_number).zfill(3) + '.jpg', my_node.image_array[i])
        
        #  Save the list of images, pressure reading and force readings to a pandas dataframe in the csv format
        header = ['image_name', 'pressure_reading_1', 'pressure_reading_2', 'force_reading_1', 'force_reading_2', 'label']
        grasp_df = pd.concat([pd.Series(image_name), pd.Series(self.pressure_reading_1), pd.Series(self.pressure_reading_2), pd.Series(self.force_reading_1), pd.Series(self.force_reading_2), pd.Series(np.repeat('square_60', len(self.image_array)))], axis=1, keys=header)
        
        combined_df_set = pd.read_csv(directory + 'Grasp_dataset_temp.csv')
        combined_df_set = pd.concat([combined_df_set, grasp_df], axis=0)
        combined_df_set.to_csv(directory + 'Grasp_dataset_temp.csv', index=False)
        
    def start(self):
        rospy.loginfo("Ready to collect data")

        while not rospy.is_shutdown():
            
            # If the inpput is empty, then one cycle of data collection is executed
            if self.command == '':
                print(f'Initial raw arduino reading with tare is: {self.tare_read_arduino()}')
                time.sleep(0.5)
                self.move_jaw(1500)
                time.sleep(2)
                self.pub.publish(5201314)
                data = self.read_arduino()
                print(f'Collected raw arduino reading with tare is: {data}')
                self.store_arduino_data(data)
                time.sleep(0.5)
                self.command = '2'
            
            # If input is '1', then the parallel jaw is moved to the closed position
            elif self.command == '1':
                self.move_jaw(1500)
            
            # If input is '2', then the parallel jaw is moved to the open position
            elif self.command == '2':
                self.move_jaw(200)
            
            # If input is '3', then the parallel jaw is disabled
            elif self.command == '3':
                self.disable_jaw()
        
            self.loop_rate.sleep()

if __name__ == '__main__':
    rospy.init_node("command_publisher", anonymous=True)
    dxl_motor = Dynamixel_motor(1, 57600, '/dev/ttyUSB0')
    my_node = Node(dxl_motor)
    my_node.start()

In [ ]:
# To save data, you can interrupt the kernel and run the following line
my_node.save_data()